# Imports

In [ ]:
import json
import requests
from bs4 import BeautifulSoup
import time
import csv
import os
import pandas as pd
import import_ipynb
import Dataprep

# Extracting data and merging CSVs

### Extracting data from saved files and merging together 
Taking the projections, draftkings, and vegas odds save files and turning them into CSVs

In [ ]:
box_df = pd.read_csv("outputs/NBABoxStats.csv")
dates = box_df['Date'].to_numpy()
date_list = Dataprep.get_dates_list(dates)
resultsDK = []
resultsOdds = []
resultsProj = []
odds_df = pd.DataFrame()
for date in date_list:
    try:
        dk_df = Dataprep.get_dk_info(date)
        resultsDK.append(dk_df)
        odds_df = Dataprep.get_betting_lines(date)
        resultsOdds.append(odds_df)
        proj_df = Dataprep.get_proj_info(date)
        resultsProj.append(proj_df)
    except:
        pass

dk_df = pd.concat(resultsDK)
dk_df.set_index('GID',inplace=True)
dk_df.sort_index(ascending=True,inplace=True)
dk_df = dk_df.reset_index()

odds_df = pd.concat(resultsOdds)
odds_df.set_index('GID', inplace=True)
odds_df.sort_index(ascending=True,inplace=True)
odds_df = odds_df.reset_index()

proj_df = pd.concat(resultsProj)
proj_df.set_index('GID',inplace=True)
proj_df.sort_index(ascending=True,inplace=True)
proj_df = proj_df.reset_index() 

dk_df.to_csv('outputs/dk_df.csv')
odds_df.to_csv('outputs/odds_df.csv')
proj_df.to_csv('outputs/proj_df.csv')

### Merging CSVs
Adding draftkings data, vegas odds, and projections data into the nba box stats csv

In [ ]:
from bs4 import BeautifulSoup
import csv
import pandas as pd
import os
import import_ipynb
import dataprep

box_df = pd.read_csv("outputs/NBABoxStats.csv")

dk_df = pd.read_csv('outputs/dk_df.csv')
box_dk = dataprep.add_dk(box_df,dk_df)

odds_df = pd.read_csv('outputs/odds_df.csv')
box_dk_odds = dataprep.add_odds(box_dk,odds_df)

proj_df = pd.read_csv('outputs/proj_df.csv')
box_dk_odds_proj = dataprep.add_proj(box_dk_odds,proj_df)

box_dk_odds_proj.to_csv('outputs/box_dk_odds_proj.csv')

# Data cleaning 

In [10]:
#Creates a column of name+date to group same-player performances by specific game
df = pd.read_csv('outputs/box_dk_odds_proj.csv')
df.dropna(inplace = True)

#df.drop_duplicates(subset=['Name','Month','Day'], keep='first',inplace=True)
df.drop(['Unnamed: 0','FGA','3PA','FTA','ORB','DRB','PF','+/-'], axis=1,inplace=True)
df = df[df['Projected Points'] > 5]

#Arrays for categorical values
mp_list = []
home_list = []
away_list = []
value_list = []
season_list = []
season_opp_list = []

i = 0
for index, each_row in df.iterrows():
    Dataprep.home_away_arr(home_list, away_list, each_row)
    Dataprep.convert_time_to_decimal(mp_list,each_row)
    value = int(2000*each_row['Points']/each_row['Salary'])/2.0
    value_list.append(value)
    Dataprep.get_season_from_date(season_list, season_opp_list, each_row)
    i=i+1

#Add columns to DataFrame
df['MP']=mp_list
df['Home']=home_list
df['Away']=away_list
df['Value'] = value_list
df['Season'] = season_list
df['Season+Opp'] = season_opp_list
    
#Sorts the data by name then goes back to the default index
df.set_index('Name',inplace=True)
df.sort_index(inplace=True)
df.reset_index(inplace=True)
#df.to_csv('Cleaned_Data_Sorted.csv')

### Grouping Performances by Player and Calculating Rolling Averages
sep_by_player() creates a dataframe for each player in our data and calculates rolling averages of the previous 5 games for DraftKings points and minutes played. Then all box stats columns for the current game are dropped.

In [11]:
df_list = Dataprep.sep_by_player(df)
df = pd.concat(df_list)
df = Dataprep.drop_box_stats_cols(df)
df.to_csv('NBAData.csv')